### 1. Init config

In [1]:
from google.cloud import bigquery
import os
import pandas as pd
import datetime

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "key.json"

client = bigquery.Client()

project_id = "zapart-data-vlille"
dataset_id = "vlille_dataset"

### 2. Stations query - dim table

In [3]:
table_name = "stations"

table_ref = client.get_table(f'{project_id}.{dataset_id}.{table_name}')
schema = table_ref.schema

for field in schema:
    print(field.name, field.field_type)


id INTEGER
nom STRING
adresse STRING
commune STRING
type STRING
latitude FLOAT
longitude FLOAT


In [4]:
sql = f"""
SELECT *
FROM `{project_id}.{dataset_id}.stations`
"""

df_stations = client.query(sql).to_dataframe()

In [5]:
df_stations.head()

,id,nom,adresse,commune,type,latitude,longitude
0,136,RUE CHAMBORD,RUE CHAMBORD,VILLENEUVE D'ASCQ,SANS TPE,50.629720,3.133873
1,270,PAVÉ DE LILLE,Rue Jean Bart,WASQUEHAL,AVEC TPE,50.664211,3.130098
2,35,LECLERC,24 PLACE MARECHAL LECLERC,LILLE,AVEC TPE,50.628990,3.043307
3,44,GAMBETTA UTRECHT,199 RUE LEON GAMBETTA,LILLE,AVEC TPE,50.629063,3.053711
4,11,NOUVEAU SIECLE,10 RUE DE PAS,LILLE,AVEC TPE,50.637340,3.060977


In [6]:
# add a column 'geo' [latitude, longitude]
df_stations['geo'] = df_stations.apply(lambda row: str(row['latitude']) + "," + str(row['longitude']), axis=1)
df_stations.head()

,id,nom,adresse,commune,type,latitude,longitude,geo
0,136,RUE CHAMBORD,RUE CHAMBORD,VILLENEUVE D'ASCQ,SANS TPE,50.629720,3.133873,"50.62972,3.133873"
1,270,PAVÉ DE LILLE,Rue Jean Bart,WASQUEHAL,AVEC TPE,50.664211,3.130098,"50.664211,3.130098"
2,35,LECLERC,24 PLACE MARECHAL LECLERC,LILLE,AVEC TPE,50.628990,3.043307,"50.62899,3.043307"
3,44,GAMBETTA UTRECHT,199 RUE LEON GAMBETTA,LILLE,AVEC TPE,50.629063,3.053711,"50.629063,3.053711"
4,11,NOUVEAU SIECLE,10 RUE DE PAS,LILLE,AVEC TPE,50.637340,3.060977,"50.63734,3.060977"


In [7]:
# save dataframe to csv
df_stations.to_csv('stations.csv', index=False)

### 3. Records query - fact table

In [3]:
table_name = "records"

table_ref = client.get_table(f'{project_id}.{dataset_id}.{table_name}')
schema = table_ref.schema

for field in schema:
    print(field.name, field.field_type)


station_id INTEGER
etat STRING
nb_velos_dispo INTEGER
nb_places_dispo INTEGER
etat_connexion STRING
derniere_maj TIMESTAMP
record_timestamp TIMESTAMP


In [10]:
sql = f"""
SELECT *
FROM 
    `{project_id}.{dataset_id}.{table_name}`
WHERE
    station_id = 25
    AND DATE(record_timestamp) = "2023-09-27"
"""

df_records = client.query(sql).to_dataframe()

# remove duplicate rows
df_records = df_records.drop_duplicates()
df_records.sort_values(by=['record_timestamp'], inplace=True)
df_records.head()

,station_id,etat,nb_velos_dispo,nb_places_dispo,etat_connexion,derniere_maj,record_timestamp
231,25,EN SERVICE,0,32,CONNECTÉ,2023-11-12 23:56:10+00:00,2023-11-13 00:00:06.624000+00:00
232,25,EN SERVICE,0,32,CONNECTÉ,2023-11-12 23:56:10+00:00,2023-11-13 00:01:01.149000+00:00
233,25,EN SERVICE,0,32,CONNECTÉ,2023-11-12 23:56:10+00:00,2023-11-13 00:02:01.485000+00:00
234,25,EN SERVICE,0,32,CONNECTÉ,2023-11-12 23:56:10+00:00,2023-11-13 00:03:01.494000+00:00
235,25,EN SERVICE,0,32,CONNECTÉ,2023-11-12 23:56:10+00:00,2023-11-13 00:04:00.899000+00:00


In [3]:
sql = f"""
SELECT station_id, nom, nb_velos_dispo, nb_places_dispo, record_timestamp, adresse, commune, latitude, longitude, type, etat, etat_connexion, derniere_maj
FROM 
    `{project_id}.{dataset_id}.stations`, 
    `{project_id}.{dataset_id}.records`
WHERE
    `{project_id}.{dataset_id}.stations`.id = `{project_id}.{dataset_id}.records`.station_id
    AND DATE(record_timestamp) = "2023-09-27"
"""

df_full = client.query(sql).to_dataframe()

In [5]:
df_full.head()

,station_id,nom,nb_velos_dispo,nb_places_dispo,record_timestamp,adresse,commune,latitude,longitude,type,etat,etat_connexion,derniere_maj
0,1,METROPOLE EUROPEENNE DE LILLE,0,0,2023-09-27 10:00:42.773000+00:00,MEL RUE DU BALLON,LILLE,50.641926,3.075992,AVEC TPE,RÉFORMÉ,DÉCONNECTÉ,2022-11-29 09:47:16+00:00
1,1,METROPOLE EUROPEENNE DE LILLE,0,0,2023-09-27 15:19:00.882000+00:00,MEL RUE DU BALLON,LILLE,50.641926,3.075992,AVEC TPE,RÉFORMÉ,DÉCONNECTÉ,2022-11-29 09:47:16+00:00
2,1,METROPOLE EUROPEENNE DE LILLE,0,0,2023-09-27 20:28:01.143000+00:00,MEL RUE DU BALLON,LILLE,50.641926,3.075992,AVEC TPE,RÉFORMÉ,DÉCONNECTÉ,2022-11-29 09:47:16+00:00
3,1,METROPOLE EUROPEENNE DE LILLE,0,0,2023-09-27 01:30:10.842000+00:00,MEL RUE DU BALLON,LILLE,50.641926,3.075992,AVEC TPE,RÉFORMÉ,DÉCONNECTÉ,2022-11-29 09:47:16+00:00
4,1,METROPOLE EUROPEENNE DE LILLE,0,0,2023-09-27 16:08:00.744000+00:00,MEL RUE DU BALLON,LILLE,50.641926,3.075992,AVEC TPE,RÉFORMÉ,DÉCONNECTÉ,2022-11-29 09:47:16+00:00


In [6]:
df_full.shape

(415871, 13)

In [27]:
df_full['geo'] = df_full.apply(lambda row: str(row['latitude']) + "," + str(row['longitude']), axis=1)

In [7]:
df_light = df_full[['station_id', 'nb_velos_dispo', 'nb_places_dispo', 'latitude', 'longitude', 'record_timestamp']]

In [8]:
df_light.head()

,station_id,nb_velos_dispo,nb_places_dispo,latitude,longitude,record_timestamp
0,1,0,0,50.641926,3.075992,2023-09-27 10:00:42.773000+00:00
1,1,0,0,50.641926,3.075992,2023-09-27 15:19:00.882000+00:00
2,1,0,0,50.641926,3.075992,2023-09-27 20:28:01.143000+00:00
3,1,0,0,50.641926,3.075992,2023-09-27 01:30:10.842000+00:00
4,1,0,0,50.641926,3.075992,2023-09-27 16:08:00.744000+00:00


In [10]:
df_light.to_csv('dataset_light_2023_09_27.csv', index=False)

In [28]:
df_full.to_csv('datataset_full.csv', index=False)